In [1]:
from gcp_processor import SilverProcessor

/Users/alexandergirardet/miniforge3/lib/python3.9/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
/Users/alexandergirardet/miniforge3/lib/python3.9/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [8]:
processor = SilverProcessor()

In [9]:
processor

In [10]:
processed_files = processor.fetch_processed_files()

all_files = processor.fetch_all_files()

remaining_files = processor.get_remaining_files(all_files, processed_files)

batches = processor.get_batch_data(remaining_files)

In [11]:
batches

[]

In [6]:
processor.process_batches(batches)

/Users/alexandergirardet/nexus_equities/transformers_gcp/nexus-transformers/final_gcp_etl/gcp_processor.py:392: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_df = full_df.append(df)
/Users/alexandergirardet/nexus_equities/transformers_gcp/nexus-transformers/final_gcp_etl/gcp_processor.py:392: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_df = full_df.append(df)
/Users/alexandergirardet/nexus_equities/transformers_gcp/nexus-transformers/final_gcp_etl/gcp_processor.py:392: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_df = full_df.append(df)
/Users/alexandergirardet/nexus_equities/transformers_gcp/nexus-transformers/final_gcp_etl/gcp_processor.py:392: FutureWarning: The frame.append method is deprecated and wil

### Individual test

In [13]:
batch = batches[1]

In [14]:
batch_data = processor.extract_batch_data(batch)
transformed_batch = processor.transform_batch_data(batch_data)

tables = processor.generate_tables(transformed_batch)

/Users/alexandergirardet/nexus_equities/transformers_gcp/nexus-transformers/final_gcp_etl/gcp_processor.py:392: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_df = full_df.append(df)
/Users/alexandergirardet/nexus_equities/transformers_gcp/nexus-transformers/final_gcp_etl/gcp_processor.py:392: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_df = full_df.append(df)
/Users/alexandergirardet/nexus_equities/transformers_gcp/nexus-transformers/final_gcp_etl/gcp_processor.py:392: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_df = full_df.append(df)
/Users/alexandergirardet/nexus_equities/transformers_gcp/nexus-transformers/final_gcp_etl/gcp_processor.py:392: FutureWarning: The frame.append method is deprecated and wil

In [15]:
results = processor.tables_to_parquet(tables)

In [29]:
def batch_to_sql(batch: list, results: list, folder_session: str) -> str:
        """Processes the results and batch ids to be inserted into the generate_sql_query function.

        Args:
            batch (list): List of s3 url strings for a given batch
            results (list): list of strings containing the results returned by wrangler to_parquet method. Which in turn contains the paths of each table destination.
            folder_session (str): String holding the time the SilverProcesser was instantiated.

        Returns:
            sql_query (str): The sql query to be inserted into reonomy_silver_processed_files postgres table.
        """

        paths = [result for result in results]

        file_id_list = [file.split("/")[-1] for file in batch]

        cleaned_file_id_list = [file.split(".")[0] for file in file_id_list]

        sql_query = generate_sql_query(cleaned_file_id_list, folder_session, paths)

        return sql_query

What I can change is that since all these files now have the same IDs. I can just give it it's own id. 

In [27]:
def generate_sql_query(
        id_list: list, folder_session: str, paths: list
    ) -> str:
        """Generates the sql query for inserting data into reonomy_silver_processed_files table.

        This function takes the list of ids of the recently processed files, as well as the folder_session and the paths of the files
        that each of those file tables were processed into and generates a SQL query.

        Args:
            id_list (list): List of ids of recently processed files.
            folder_session (str): String holding the time the SilverProcesser was instantiated.
            paths (list): List of strings containing the destination paths of each table within a file batch.

        Returns:
            sql_query (str): The sql query to be inserted into reonomy_silver_processed_files postgres table.
        """

        descriptions_path = paths[0]
        sales_path = paths[1]
        tenants_path = paths[2]
        shapes_path = paths[3]
        contacts_path = paths[4]
        taxes_path = paths[5]

        tuples_to_insert = []

        for file_id in id_list:
            file_tuple = (
                file_id,
                folder_session,
                descriptions_path,
                sales_path,
                tenants_path,
                shapes_path,
                contacts_path,
                taxes_path,
            )
            tuples_to_insert.append(file_tuple)

        tuple_string = str(tuples_to_insert)

        sql_query = f"INSERT INTO reonomy_silver_processed_files(file_id, processed_date, descriptions_path, sales_path, tenants_path, shapes_path, contacts_path, taxes_path) VALUES {tuple_string[1:-1]}"

        return sql_query

In [28]:
batch_to_sql(batch, results, processor.folder_session)

"INSERT INTO reonomy_silver_processed_files(file_id, processed_date, descriptions_path, sales_path, tenants_path, shapes_path, contacts_path, taxes_path) VALUES ('314e28d4-7551-42db-add5-de74cd8de3a4', '2023-1-7-14-5', 'test_parquet_1/descriptions/fcdbcc8f-8312-42b1-9c32-09fcd0793e74.snappy.parquet', 'test_parquet_1/sales/fcdbcc8f-8312-42b1-9c32-09fcd0793e74.snappy.parquet', 'test_parquet_1/tenants/fcdbcc8f-8312-42b1-9c32-09fcd0793e74.snappy.parquet', 'test_parquet_1/shapes/fcdbcc8f-8312-42b1-9c32-09fcd0793e74.snappy.parquet', 'test_parquet_1/contacts/fcdbcc8f-8312-42b1-9c32-09fcd0793e74.snappy.parquet', 'test_parquet_1/taxes/fcdbcc8f-8312-42b1-9c32-09fcd0793e74.snappy.parquet'), ('3fe4b946-6ee7-4191-837c-41ed67936efb', '2023-1-7-14-5', 'test_parquet_1/descriptions/fcdbcc8f-8312-42b1-9c32-09fcd0793e74.snappy.parquet', 'test_parquet_1/sales/fcdbcc8f-8312-42b1-9c32-09fcd0793e74.snappy.parquet', 'test_parquet_1/tenants/fcdbcc8f-8312-42b1-9c32-09fcd0793e74.snappy.parquet', 'test_parquet_1/